### コーパスの中身をkmeansでクラスタリング

In [1]:
from collections import defaultdict

from gensim import corpora
from gensim import models
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
model = KeyedVectors.load('output/word2vec_ramen_model.model')

In [3]:
max_vocab = 30000
vocab = list(model.wv.key_to_index.keys())[:max_vocab]
vectors = [model.wv[word] for word in vocab]

In [5]:
len(vectors)

6887

In [6]:
# クラスター数はこちらで任意の値を定める
n_clusters = 6
kmeans_model = KMeans(n_clusters=n_clusters, verbose=0, random_state=42)
kmeans_model.fit(vectors)

KMeans(n_clusters=6, random_state=42)

In [7]:
cluster_labels = kmeans_model.labels_
cluster_to_words = defaultdict(list)
for cluster_id, word in zip(cluster_labels, vocab):
    cluster_to_words[cluster_id].append(word)

In [9]:
for words in cluster_to_words.values():
    print(words[:20])

['スープ', 'よう', '醤油', '美味しい', 'これ', '感じ', '良い', '煮干し', '旨味', '濃厚', 'いい', '出汁', '香り', '味わい', 'つけ', '風味', '良く', '魚介', 'なく', 'バランス']
['ラーメン', 'こと', 'そう', 'もの', 'ない', 'それ', '好き', 'ところ', 'ここ', '一杯', '美味しかっ', '美味しく', '高い', 'みたい', 'ため', '多い', '自分', '以上', '通り', 'なし']
['チャーシュー', 'メンマ', 'トッピング', 'ワンタン', 'ネギ', '食感', '海苔', '具材', '調理', 'シンプル', '味付け', '相性', '0種類', 'タレ', '低温', '柔らかく', '柔らか', '玉子', 'もやし', 'ロース']
['0円', 'そば', '注文', 'つけ麺', '特製', '中華そば', 'メニュー', 'らーめん', '今回', '提供', '普通', 'ご飯', '限定', '味噌', '醤油ラーメン', '塩ラーメン', '¥0', '0g', 'オーダー', '本日']
['0分', '0時', '店内', 'カウンター', '食券', '到着', '待ち', '行列', '購入', '券売機', '入店', '時間', '店主', '開店', '店員', '着席', '雰囲気', '丁寧', '案内', 'お客さん']
['さん', '訪問', 'こちら', '人気', '平日', 'ごちそうさま', 'ラーメン屋', '今日', '徒歩', '食べログ', '名店', '近く', 'オープン', '場所', '百名', 'ランチ', '営業', '評価', '東京', '仕事']


In [10]:
def change_dict_key(d, old_key, new_key, default_value=None):
    d[new_key] = d.pop(old_key, default_value)

In [11]:
# 各クラスターの解釈を決定
change_dict_key(cluster_to_words, 0, 'スープに関するワード')
change_dict_key(cluster_to_words, 1, 'その他のワード')
change_dict_key(cluster_to_words, 2, 'トッピングに関するワード')
change_dict_key(cluster_to_words, 3,  'ラーメンに関するワード')
change_dict_key(cluster_to_words, 4, '店内に関するワード')
change_dict_key(cluster_to_words, 5, '店舗に関するワード')

In [13]:
df = pd.DataFrame.from_dict(cluster_to_words, orient="index").T
df = df.iloc[:, [3, 0, 2, 4, 5, 1]]
df.head()

,ラーメンに関するワード,スープに関するワード,トッピングに関するワード,店内に関するワード,店舗に関するワード,その他のワード
0,スープ,ラーメン,チャーシュー,さん,0分,0円
1,よう,こと,メンマ,訪問,0時,そば
2,醤油,そう,トッピング,こちら,店内,注文
3,美味しい,もの,ワンタン,人気,カウンター,つけ麺
4,これ,ない,ネギ,平日,食券,特製


### TF-IDFで文章における特徴的な単語を抽出

In [20]:
from gensim import corpora
from gensim import models

In [18]:
# 口コミからレコメンドに使用する情報を絞る
# 試行錯誤して決定するべし!!
ramen_words = cluster_to_words['ラーメンに関するワード']
soup_words = cluster_to_words['スープに関するワード']
topping_words = cluster_to_words['トッピングに関するワード']

ramen_words.extend(soup_words)
ramen_words.extend(topping_words)
key_words = ramen_words

len(key_words)

15019

In [19]:
with open('output/ramen_corpus.txt', 'r', encoding='utf-8') as f:
    trainings = []
    for i, data in enumerate(f):
        # 不要な文字を取り除いて単語を取り出す
        words = data.replace("'", '').replace('[', '').replace(']', '').replace(' ', '').replace('\n', '').split(",")
        trainings.append([i for i in words if i in key_words])

In [21]:
# 単語->id変換の辞書作成
dictionary = corpora.Dictionary(trainings)

In [22]:
# textsをcorpus化
corpus = list(map(dictionary.doc2bow, trainings))

In [23]:
# tfidf modelの生成
test_model = models.TfidfModel(corpus)

In [24]:
# corpusへのモデル適用
corpus_tfidf = test_model[corpus]

In [25]:
# id->単語へ変換
texts_tfidf = []    # id -> 単語表示に変えた文書ごとのTF-IDF
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary[word[0]], word[1]])
    texts_tfidf.append(text_tfidf)

In [28]:
from operator import itemgetter
import pickle

In [27]:
texts_tfidf_sorted_top20 = [] 

#TF-IDF値を高い順に並び替え上位単語20個に絞る。
for i in range(len(texts_tfidf)):
    soted = sorted(texts_tfidf[i], key=itemgetter(1),reverse=True)
    soted_top20 = soted[:20]
    word_list = []
    for k in range(len(soted_top20)):
        word = soted_top20[k][0]
        word_list.append(word)
    texts_tfidf_sorted_top20.append(word_list)

In [44]:
# 結果をデータフレームに追加
store_df = pd.read_csv('output/ramen_store.csv')
review_df = pd.read_csv('output/ramen_review.csv')
df = pd.merge(store_df, review_df)

print('columns:\n', df_ramen.columns)
print('\nshape: ', df_ramen.shape)

columns:
 Index(['ID', 'name', 'score', 'review_count', 'review',
       'texts_tfidf_sorted_top20'],
      dtype='object')

shape:  (200, 6)


In [85]:
df_ramen = df.groupby(['ID', 'name', 'score', 'review_count'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)
df_ramen['texts_tfidf_sorted_top20'] = texts_tfidf_sorted_top20

df_ramen_texts_tfidf_sorted_top20 = df_ramen.iloc[:,[0, 1, 5]].reset_index(drop=True)
df_ramen_texts_tfidf_sorted_top20.head()

,ID,name,texts_tfidf_sorted_top20
0,1,らぁ麺や 嶋,"[ワンタン, 炭火, 海老, 昆布, 佐野実, だれ, 焼き豚, 炙り焼, 佐野, 藻塩, ..."
1,2,Homemade Ramen 麦苗,"[黒毛和牛, 火山, 深谷, spf豚, spf, 実家, ワンタン, 手入れ, 鰹節, オ..."
2,3,手打式超多加水麺 ののくら,"[ワンタン, 手打, 三つ葉, 鶏胸肉, 生醤油, うどん, 大葉, 0%, 紫蘇, 形成,..."
3,4,麺尊 RAGE,"[しゃも, 卵黄, 醤油漬け, 素地, レモン, 浸透, シャモ, 燻製, 鶏油, 咀嚼, ..."
4,5,宍道湖しじみ中華蕎麦 琥珀,"[しじみ, 島根県, シジミ, ワンタン, 琥珀, バラ, 雑色, コハク酸, 菅野, 焼き..."


In [86]:
# 上の結果をダンプ
pickle.dump(df_ramen_texts_tfidf_sorted_top20, open('output/df_ramen_texts_tfidf_sorted_top20', 'wb'))

### お店間の類似度を計算

In [87]:
from itertools import product

In [88]:
f = open('output/df_ramen_texts_tfidf_sorted_top20','rb')
store_df = pickle.load(f)
store_cross = []
for ids in product(store_df['ID'], repeat=2):
    store_cross.append(ids)

In [89]:
store_cross_df = pd.DataFrame(store_cross, columns=['id_x', 'id_y'])
store_cross_df.head()

,id_x,id_y
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


In [95]:
store_cross_detail = store_cross_df.merge(
    store_df[['ID','name','texts_tfidf_sorted_top20']], how='inner', left_on='id_x', right_on='ID'
).drop(columns='ID').merge(
    store_df[['ID','name','texts_tfidf_sorted_top20']], how='inner', left_on='id_y', right_on='ID'
).drop(columns='ID')

store_cross_detail = store_cross_detail[store_cross_detail['id_x'].isin(store_df['ID'].loc[0:50])]
store_cross_detail = store_cross_detail.reset_index(drop=True).sort_values(['id_x'])

store_cross_detail.head()

,id_x,id_y,name_x,texts_tfidf_sorted_top20_x,name_y,texts_tfidf_sorted_top20_y
0,1,1,らぁ麺や 嶋,"[ワンタン, 炭火, 海老, 昆布, 佐野実, だれ, 焼き豚, 炙り焼, 佐野, 藻塩, ...",らぁ麺や 嶋,"[ワンタン, 炭火, 海老, 昆布, 佐野実, だれ, 焼き豚, 炙り焼, 佐野, 藻塩, ..."
5508,1,121,らぁ麺や 嶋,"[ワンタン, 炭火, 海老, 昆布, 佐野実, だれ, 焼き豚, 炙り焼, 佐野, 藻塩, ...",食堂七彩,"[かた焼, きそば, 海老, クラゲ, 風麻, 中華屋, 紹興酒, 白菜, 味の素, 息子,..."
5559,1,120,らぁ麺や 嶋,"[ワンタン, 炭火, 海老, 昆布, 佐野実, だれ, 焼き豚, 炙り焼, 佐野, 藻塩, ...",MENSHO,"[煮干し, こんにゃく, 穂先, 三つ葉, 煮干, ごま油, 高菜, 三河, 三河屋, 特長..."
5610,1,119,らぁ麺や 嶋,"[ワンタン, 炭火, 海老, 昆布, 佐野実, だれ, 焼き豚, 炙り焼, 佐野, 藻塩, ...",RAMEN GOTTSU,"[ワカメ, 貝出, タケノコ, 豚野郎, 三つ葉, アサリ, 浅利, フライドオニオン, 長..."
5661,1,118,らぁ麺や 嶋,"[ワンタン, 炭火, 海老, 昆布, 佐野実, だれ, 焼き豚, 炙り焼, 佐野, 藻塩, ...",新世界菜館,"[煮干し, 比内, 自家製, エグミ, 九十九里, 自信, パッツンパッツン, オイラ, 調..."


#### ラーメン店xに対してラーメン店yの類似度を算出

In [59]:
import itertools
from tqdm import tqdm 

In [52]:
#コサイン類似度を算出する関数を定義
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [96]:
# word2vecのモデルを読み込み
import numpy as np
from gensim.models import word2vec

word2vec_ramen_model = word2vec.Word2Vec.load("output/word2vec_ramen_model.model")

In [97]:
# cossimだけの組み合わせ（同じワード同士の組みあわせがでてくるため）
# 2次元を１次元にする　setが重複を削除てきなやつ。
uniq_words = list(set(itertools.chain.from_iterable(store_df['texts_tfidf_sorted_top20'].values)))
scores = {}
for word1, word2 in product(uniq_words, repeat=2):
    scores[(word1, word2)] = cos_sim(word2vec_ramen_model.wv[word1],
                                     word2vec_ramen_model.wv[word2]) 

In [98]:
avg_avg_scores = []
for i in tqdm(range(len(store_cross_detail['texts_tfidf_sorted_top20_x']))):
    avg_scores = []
    for j in range(len(store_cross_detail['texts_tfidf_sorted_top20_x'][i])):
        word_cross_scores = []
        word_a = store_cross_detail['texts_tfidf_sorted_top20_x'][i][j]
        for k in range(len(store_cross_detail['texts_tfidf_sorted_top20_y'][i])):
            word_b = store_cross_detail['texts_tfidf_sorted_top20_y'][i][k]
            # 単語間のスコアを出す。
            score = scores[(word_a, word_b)]
            word_cross_scores.append(score)
        # 20個の単語間スコアの平均値
        avg_scores.append(np.mean(word_cross_scores))
        # 20個の単語間スコアの平均値の平均値
    avg_avg_scores.append(np.mean(avg_scores))
store_cross_detail.insert(6, 'avg_cos_sim_rate', avg_avg_scores)  

100%|██████████| 10200/10200 [00:20<00:00, 494.19it/s]


In [104]:
# 類似度が高いラーメン屋を高い順に表示
keyword = 'ののくら'
store_cross_detail = store_cross_detail.sort_values(['id_x', 'avg_cos_sim_rate'], ascending=[True, False])
df_sim_x = store_cross_detail[store_cross_detail['name_x'].str.contains(keyword)]
df_sim_x.reset_index(drop=True)
df_sim_x.head()

,id_x,id_y,name_x,texts_tfidf_sorted_top20_x,name_y,texts_tfidf_sorted_top20_y,avg_cos_sim_rate
8570,3,168,手打式超多加水麺 ののくら,"[ワンタン, 手打, 三つ葉, 鶏胸肉, 生醤油, うどん, 大葉, 0%, 紫蘇, 形成,...",楽観 NISHIAZABU GOLD,"[八王子ラーメン, 玉ネギ, 玉ねぎ, 巧真, ラード, みじん, 一面, 細か, タマネギ...",0.705828
1124,3,26,手打式超多加水麺 ののくら,"[ワンタン, 手打, 三つ葉, 鶏胸肉, 生醤油, うどん, 大葉, 0%, 紫蘇, 形成,...",カネキッチン ヌードル,"[みょうが, 黒木, 生姜, たけ, エレガント, ドライトマト, 鶏団子, ワンタン, つ...",0.699934
4796,3,103,手打式超多加水麺 ののくら,"[ワンタン, 手打, 三つ葉, 鶏胸肉, 生醤油, うどん, 大葉, 0%, 紫蘇, 形成,...",大島,"[もやし, ワンタン, 町中華, フライドオニオン, 中華屋, ニラ, ニンジン, モヤシ,...",0.699866
6173,3,107,手打式超多加水麺 ののくら,"[ワンタン, 手打, 三つ葉, 鶏胸肉, 生醤油, うどん, 大葉, 0%, 紫蘇, 形成,...",麺屋海神 新宿店,"[鯛の刺身, 鯛の切り身, 柚子, お麩, 茶漬, 柚子の皮, 切り身, スーフ, 穂先, ...",0.696450
9233,3,198,手打式超多加水麺 ののくら,"[ワンタン, 手打, 三つ葉, 鶏胸肉, 生醤油, うどん, 大葉, 0%, 紫蘇, 形成,...",田中そば店 秋葉原店,"[コーン, ファイヤー, もやし, オロチョン, 赤味噌, 開花, 東京スタイル, 札幌, ...",0.693576


In [105]:
# 正規化
def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result

In [106]:
b = df_sim_x['avg_cos_sim_rate']
c = min_max(b.values)
df_sim_x.insert(7, '正規化', c)

In [110]:
df_sim_x = df_sim_x[['id_y', 'name_y', 'avg_cos_sim_rate', '正規化',]]
df_sim_x.head(20)

,id_y,name_y,avg_cos_sim_rate,正規化
8570,168,楽観 NISHIAZABU GOLD,0.705828,1.000000
1124,26,カネキッチン ヌードル,0.699934,0.970426
4796,103,大島,0.699866,0.970087
6173,107,麺屋海神 新宿店,0.696450,0.952948
9233,198,田中そば店 秋葉原店,0.693576,0.938529
4031,89,神名備,0.687552,0.908305
1226,27,ラーメン 健やか,0.686824,0.904654
3113,64,中華ソバ ちゃるめ,0.685208,0.896548
257,6,中華そば しば田,0.682388,0.882398
8060,171,中華そば 吾衛門,0.681949,0.880193
